In [ ]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable



class DecentNet(nn.Module):
    
    # for freezing in train code: retinanet.backbone.requires_grad_(False)
        
    def __init__(self, num_classes=3):
        super(DecentNet_v1, self).__init__()
        
        print("init decentnet start")

        self.num_classes = num_classes
        
        # prepare early blocks list
        ckpt_early_blocks = os.listdir(ckpt_early_blocks_path)
        self.early_blocks = nn.ModuleList([])
        early_block_116 = None
        for i, early_block in enumerate(ckpt_early_blocks):            
            early_block_116 = DecentBlock_Shuffle_EarlyBlock(ckpt_early_blocks_path, early_block, out_channels=5)
            self.early_blocks.append(early_block_116) # 116 output filters
                
        # prepare early fusion (optional)
        self.early_fusion_module = None
        
        # prepare late blocks list (optional)
        self.late_blocks = nn.ModuleList([])
        self.late_blocks.append(DecentBlock_ResNet_LateBlock())
        
        # prepare late fusion (optional)
        self.late_fusion_module = None
        
        # prepare head blocks list
        self.head_blocks = nn.ModuleList([])
        self.late_blocks.append(DecentBlock_ResNet_LateBlock())

        print("init decentnet done")
        
        
    def forward(self, x):
        
        # early block(s)
        block_outputs = []
        for i, block in enumerate(self.early_blocks):
            block_output = block(x)
            block_outputs.append(block_output)
            
        # concat + fusion
        if block_outputs:
            x = torch.cat(block_outputs, dim=1)
            x = self.early_fusion_module(x)
        
        # late block(s)
        block_outputs = []
        for i, block in enumerate(self.late_blocks):
            block_output = block(x)
            block_outputs.append(block_output)
            
        # concat + fusion
        if block_outputs:
            x = torch.cat(block_outputs, dim=1)
            x = self.late_fusion_module(x)
        
        # head block(s) (multi-task)            
        model_outputs = {}
        for i, block in enumerate(self.head_blocks):
            # model output of head = pass feature vector through head
            model_outputs[type(head).__name__] = fc_layer(x)
        
        return model_outputs
        